# Crypto Altcoin Analysis
This workbook will walk through extracting data from Coinbase on all altcoins available with a USD pairing, then clean data and do some performance and correlation analysis.



In [1]:
# Imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import numpy as np
import pandas as pd
#from Historic_Crypto import HistoricalData
import hvplot.pandas
#import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import csv
import cbpro
import requests
import time
#wallaawallabingbangalangalang

ModuleNotFoundError: No module named 'panel'

In [2]:
# Setup Public Client for Coinbase
public_client = cbpro.PublicClient()

In [3]:
# Pull a list of all currencies (Assets and pairings)
currency_response_list = public_client.get_currencies()

In [8]:
# Pull only the assets that are marked as crypto

# Create empty list of cryptos
crypto_list = []

# If the asset is labeled as a crypto add to crypto list
for asset in currency_response_list:
    if asset['details']['type'] == 'crypto':
        crypto_list.append(asset['id'])
crypto_list.sort()
print(crypto_list)
print(len(crypto_list))

['1INCH', 'AAVE', 'ADA', 'ALGO', 'AMP', 'ANKR', 'ATOM', 'BAL', 'BAND', 'BAT', 'BCH', 'BNT', 'BOND', 'BTC', 'CGLD', 'CHZ', 'CLV', 'COMP', 'CRV', 'CTSI', 'CVC', 'DAI', 'DASH', 'DNT', 'DOGE', 'DOT', 'ENJ', 'EOS', 'ETC', 'ETH', 'FARM', 'FET', 'FIL', 'FORTH', 'GNT', 'GRT', 'GTC', 'ICP', 'KEEP', 'KNC', 'LINK', 'LOOM', 'LPT', 'LRC', 'LTC', 'MANA', 'MASK', 'MATIC', 'MIR', 'MKR', 'MLN', 'NKN', 'NMR', 'NU', 'OGN', 'OMG', 'OXT', 'PAX', 'POLY', 'QNT', 'REN', 'REP', 'RLC', 'RLY', 'SHIB', 'SKL', 'SNX', 'SOL', 'STORJ', 'SUSHI', 'TRB', 'UMA', 'UNI', 'USDC', 'USDT', 'WBTC', 'XLM', 'XRP', 'XTZ', 'YFI', 'ZEC', 'ZRX']
82


In [11]:
# Get a list of all Coinbase Products
all_products_list = public_client.get_products()
print(len(all_products_list))
#all_products_list

244


In [26]:
# Find product ids that have a USD quote_currency and create a list

# Create Empty List for USD paired products
usd_product_id_list = []
for product in all_products_list:
    if product['quote_currency'] == 'USD':
        usd_product_id_list.append(product['id'])
usd_product_id_list.sort()
cleaned_product_list = []
[cleaned_product_list.append(x) for x in usd_product_id_list if x not in cleaned_product_list]
print(cleaned_product_list)
print(type(cleaned_product_list))
print(len(cleaned_product_list))

['1INCH-USD', 'AAVE-USD', 'ADA-USD', 'ALGO-USD', 'AMP-USD', 'ANKR-USD', 'ATOM-USD', 'BAL-USD', 'BAND-USD', 'BAT-USD', 'BCH-USD', 'BNT-USD', 'BOND-USD', 'BTC-USD', 'CGLD-USD', 'CHZ-USD', 'CLV-USD', 'COMP-USD', 'CRV-USD', 'CTSI-USD', 'DAI-USD', 'DASH-USD', 'DOGE-USD', 'DOT-USD', 'ENJ-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'FARM-USD', 'FET-USD', 'FIL-USD', 'FORTH-USD', 'GRT-USD', 'GTC-USD', 'ICP-USD', 'KEEP-USD', 'KNC-USD', 'LINK-USD', 'LPT-USD', 'LRC-USD', 'LTC-USD', 'MANA-USD', 'MASK-USD', 'MATIC-USD', 'MIR-USD', 'MKR-USD', 'MLN-USD', 'NKN-USD', 'NMR-USD', 'NU-USD', 'OGN-USD', 'OMG-USD', 'OXT-USD', 'PAX-USD', 'POLY-USD', 'QNT-USD', 'REN-USD', 'REP-USD', 'RLC-USD', 'RLY-USD', 'SKL-USD', 'SNX-USD', 'SOL-USD', 'STORJ-USD', 'SUSHI-USD', 'TRB-USD', 'UMA-USD', 'UNI-USD', 'USDT-USD', 'WBTC-USD', 'XLM-USD', 'XTZ-USD', 'YFI-USD', 'ZEC-USD', 'ZRX-USD']
<class 'list'>
75


In [41]:
# Make calls to the Products endpoint for each product_Id and pull candlestick data for the last 300 weeks
# GET https://api.pro.coinbase.com/products/<product-id>/candles
# Reponses contain the following metrics in a list of lists.
# [0] time bucket start time
# [1] low lowest price during the bucket interval
# [2] high highest price during the bucket interval
# [3] open opening price (first trade) in the bucket interval
# [4] close closing price (last trade) in the bucket interval
# [5] volume volume of trading activity during the bucket interval
# columns1 = ['Date'] + usd_product_id_list
main_df = pd.DataFrame()

# Apparently only some of these product ID's have data going back to 2019 - these are those
good_list = ['ALGO-USD', 'BCH-USD', 'BTC-USD', 'DASH-USD', 'EOS-USD', 'ETC-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'REP-USD', 'UMA-USD', 'XLM-USD', 'XTZ-USD', 'ZRX-USD']

for item in good_list:
    historical = pd.DataFrame(public_client.get_product_historic_rates(item, start='2019-01-01T00:00:00-06:00', end='2019-10-19T00:00:00-05:00', granularity=86400))
    historical.columns=["Date","Open","High","Low",item,"Volume"]
    historical['Date'] = pd.to_datetime(historical['Date'], unit='s')
    historical.set_index('Date', inplace=True)
    historical.sort_values(by='Date', ascending=True, inplace=True)
    historical = historical.drop(['Open', 'High', 'Low', 'Volume'], axis=1)
    main_df = pd.concat([main_df, historical], axis=1)
    time.sleep(10)
print(main_df)

    

Data for ALGO-USD
Data for BCH-USD
Data for BTC-USD
Data for DASH-USD
Data for EOS-USD
Data for ETC-USD
Data for ETH-USD
Data for LINK-USD
Data for LTC-USD
Data for REP-USD
Data for UMA-USD
Data for XLM-USD
Data for XTZ-USD
Data for ZRX-USD
            ALGO-USD  BCH-USD   BTC-USD  DASH-USD  EOS-USD  ETC-USD  ETH-USD  \
Date                                                                           
2019-01-02       NaN   170.00   3890.79       NaN      NaN    5.350   153.50   
2019-01-03       NaN   159.88   3787.57       NaN      NaN    5.030   147.23   
2019-01-04       NaN   159.39   3820.82       NaN      NaN    5.220   153.24   
2019-01-05       NaN   158.41   3798.62       NaN      NaN    5.110   154.21   
2019-01-06       NaN   164.70   4040.99       NaN      NaN    5.450   155.92   
2019-01-07       NaN   159.71   4006.01       NaN      NaN    5.010   150.62   
2019-01-08       NaN   159.79   3993.86       NaN      NaN    4.970   148.97   
2019-01-09       NaN   158.70   4004.12